<a href="https://colab.research.google.com/github/Sihan-A/PyTorch/blob/main/PyTorch_Lightning/03_Time_Series_Valkov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[TouTube](https://www.youtube.com/watch?v=jR0phoeXjrc&t=249s)

In [1]:
!pip install --quiet pytorch-lightning==1.2.5

     |████████████████████████████████| 826 kB 14.8 MB/s 
     |████████████████████████████████| 282 kB 72.4 MB/s 
     |████████████████████████████████| 119 kB 57.3 MB/s 
     |████████████████████████████████| 269 kB 74.5 MB/s 
     |████████████████████████████████| 829 kB 73.2 MB/s 
     |████████████████████████████████| 1.3 MB 36.3 MB/s 
     |████████████████████████████████| 142 kB 70.9 MB/s 
     |████████████████████████████████| 294 kB 95.4 MB/s 


In [2]:
!pip install --quiet tqdm==4.59.0

     |████████████████████████████████| 74 kB 1.8 MB/s 


In [3]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE","#FFDD00","#FF7D00","#FF006D","#ADFF02","#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] =12,8
tqdm.pandas()

In [5]:
pl.seed_everything(42)

Global seed set to 42


42

## Load Data

In [6]:
!gdown --id 174pzX55XaNzWgjZSgX6Us13fnBvPCca6

Downloading...
From: https://drive.google.com/uc?id=174pzX55XaNzWgjZSgX6Us13fnBvPCca6
To: /content/Binance_BTCUSDT_minute.csv
37.2MB [00:00, 171MB/s] 


In [7]:
df = pd.read_csv("Binance_BTCUSDT_minute.csv", parse_dates=["date"])
df = df.sort_values(by="date").reset_index(drop=True)
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284


In [8]:
df.shape

(273288, 10)

## Preprocessing

In [9]:
df.close

0         10316.00
1         10319.50
2         10319.34
3         10319.00
4         10319.47
            ...   
273283    57816.93
273284    57930.28
273285    58049.58
273286    58215.14
273287    58331.16
Name: close, Length: 273288, dtype: float64

In [10]:
df["prev_close"] = df.shift(1)["close"]

In [11]:
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,prev_close
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342,NaN
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256,10316.00
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193,10319.50
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187,10319.34
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284,10319.00


In [12]:
df["close_change"]=df.progress_apply(
    lambda row: 0 if np.isnan(row.prev_close) else row.close - row.prev_close,
    axis=1
)

  0%|          | 0/273288 [00:00<?, ?it/s]

In [13]:
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,prev_close,close_change
0,1599856800000,2020-09-11 20:40:00,BTC/USDT,10317.00,10317.01,10314.36,10316.00,32.997684,340398.477553,342,NaN,0.00
1,1599856860000,2020-09-11 20:41:00,BTC/USDT,10316.00,10321.09,10316.00,10319.50,16.328427,168497.705251,256,10316.00,3.50
2,1599856920000,2020-09-11 20:42:00,BTC/USDT,10319.50,10320.31,10318.49,10319.34,12.123487,125108.931436,193,10319.50,-0.16
3,1599856980000,2020-09-11 20:43:00,BTC/USDT,10319.34,10320.14,10318.89,10319.00,13.548103,139806.963489,187,10319.34,-0.34
4,1599857040000,2020-09-11 20:44:00,BTC/USDT,10318.99,10319.72,10316.58,10319.47,14.466964,149266.708094,284,10319.00,0.47


In [14]:
rows =[]
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    row_data = dict(
        day_of_week=row.date.dayofweek,
        day_of_month=row.date.day,
        week_of_year=row.date.week,
        month=row.date.month,
        open=row.open,
        high=row.high,
        low=row.low,
        close_change=row.close_change,
        close=row.close
    )
    rows. append(row_data)
features_df = pd.DataFrame(rows)

  0%|          | 0/273288 [00:00<?, ?it/s]

In [15]:
features_df.shape

(273288, 9)

In [16]:
features_df.head()

,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,4,11,37,9,10317.00,10317.01,10314.36,0.00,10316.00
1,4,11,37,9,10316.00,10321.09,10316.00,3.50,10319.50
2,4,11,37,9,10319.50,10320.31,10318.49,-0.16,10319.34
3,4,11,37,9,10319.34,10320.14,10318.89,-0.34,10319.00
4,4,11,37,9,10318.99,10319.72,10316.58,0.47,10319.47


In [17]:
train_size = int(len(features_df)*0.9)
train_size

245959

In [18]:
train_df, test_df = features_df[:train_size], features_df[train_size+1:]
train_df.shape, test_df.shape

((245959, 9), (27328, 9))

In [19]:
scalar = MinMaxScaler(feature_range=(-1,1))
scalar = scalar.fit(train_df)

In [23]:
train_df = pd.DataFrame(
    scalar.transform(train_df),
    index=train_df.index,
    columns=train_df.columns,
)
test_df = pd.DataFrame(
    scalar.transform(test_df),
    index=test_df.index,
    columns=test_df.columns,
)

In [24]:
train_df.head()

,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,0.333333,-0.333333,0.384615,0.454545,-0.993056,-0.993463,-0.992628,0.242895,-0.993097
1,0.333333,-0.333333,0.384615,0.454545,-0.993097,-0.993294,-0.992559,0.244938,-0.992952
2,0.333333,-0.333333,0.384615,0.454545,-0.992952,-0.993326,-0.992456,0.242802,-0.992959
3,0.333333,-0.333333,0.384615,0.454545,-0.992959,-0.993333,-0.992439,0.242697,-0.992973
4,0.333333,-0.333333,0.384615,0.454545,-0.992973,-0.993351,-0.992535,0.243169,-0.992953
